Tải những thư viện cần thiết

In [1]:
!pip install tensorflow_decision_forests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 70.3 MB/s eta 0:00:00


In [2]:
!pip install wurlitzer
!pip install joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import thư viện

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
import pickle



Đọc dataset HomeC.csv gán nó vào một dataframe

In [5]:
dataset_df= pd.read_csv("/content/HomeC.csv")

# Tiếp tục đọc dữ liệu còn lại



# Kết hợp các dữ liệu lại với nhau



# Display the first 3 examples.
dataset_df.head(3)

<ipython-input-5-c29ddcc56c7f>:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_df= pd.read_csv("/content/HomeC.csv")


,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],...,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
1,1451624401,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
2,1451624402,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,...,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0


Xử lý trùng dữ liệu và dữ liệu bị thiếu

In [7]:
dataset_df = dataset_df.fillna(dataset_df.mean())  # Thay thế các giá trị null bằng giá trị trung bình của cột tương ứng
dataset_df = dataset_df.drop_duplicates()


<ipython-input-7-ae9c9cf67da7>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  dataset_df = dataset_df.fillna(dataset_df.mean())  # Thay thế các giá trị null bằng giá trị trung bình của cột tương ứng


Gán Features là input và target là output ở đây tụi em dự đoán tốc độ gió dựa trên temp,humi

In [8]:
features = dataset_df[['temperature', 'humidity']]

In [9]:
target = dataset_df[['windSpeed']]

In [ ]:
Kiêm tra xem có giá trị nào null không

In [10]:
target.isnull().sum(),target.shape

(windSpeed    0
 dtype: int64,
 (84590, 1))

In [11]:
features.isnull().sum(),features.shape

(temperature    0
 humidity       0
 dtype: int64,
 (84590, 2))

chia tập ra làm 2 phần train và test .tập test 1/5 độ lớn của dataframe ban đầu và train chiểm 4/5

In [12]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2,random_state=42)


rf được định nghĩa là thuật toán Random forrest với 100 decision trees với độ sâu tối đa là 10, seed ngẫu nhiên là 42. độ sâu và số lượng cây sẽ bị ảnh hưởng bởi số cột và số dòng dữ liệu ở đâu vào. độ sâu sẽ học được những nguyên tắc của dữ liệu trong một cột còn số lượng cây sẽ học những nguyên tắc của cột trong một hàng

In [13]:
# Bước 3: Xây dựng mô hình Random Forest
rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)  # Cấu hình các tham số của mô hình
rf.fit(X_train, y_train)  # Huấn luyện mô hình

<ipython-input-13-62aa1c735b18>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)  # Huấn luyện mô hình


RandomForestRegressor(max_depth=10, random_state=42)

save mô hình thành file joblib để có thể tái sử dụng

In [15]:
joblib.dump(rf, 'random_forest_model.joblib')

đánh giá mức độ lệch của mô hình và tính độ chính xác của mô hình này ở đây ta có mo hình này có độ chính xác là 0.785

In [14]:
# Bước 4: Đánh giá mô hình
y_pred = rf.predict(X_test)  # Dự đoán kết quả trên tập kiểm tra
mae = mean_absolute_error(y_test, y_pred)  # Đánh giá độ chính xác bằng mean squared error
print("Mean Absolute Error:", mae)
r2 = r2_score(y_test, y_pred)
print(r2)

Mean Absolute Error: 1.3780790672897016
0.7850696520407925


load lại mô hình và dự đoán thử

In [16]:
import numpy as np
model = joblib.load('/content/random_forest_model.joblib')
x = np.array([[34,90]])
model.predict(x)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([1.11])